In [74]:
import tensorflow

print(tensorflow.__version__)

2.9.2


In [75]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

# 데이터 전처리

In [76]:
file_path = '/content/drive/MyDrive/아이펠 데이터/exp10/fra-eng/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
112377,She's just putting up a front.,Elle fait juste semblant.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
12092,Tom felt great.,Tom se sentait très bien.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
41031,Do you have a pencil?,As-tu un crayon ?,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
156314,The president is getting into the car.,Le président embarque dans la voiture.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
85891,Does this make me look fat?,Ceci me fait-il paraître grosse ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [77]:
# 세번째 열은 불필요하므로 제거하고, 훈련 데이터는 3.3만개의 샘플로 줄이겠습니다.

lines = lines[['eng', 'fra']][:33000] # 3.3만개 샘플 사용
lines.sample(5)

,eng,fra
20343,Show us the room.,Montrez-nous la pièce.
1036,I crashed.,Je suis tombée.
2374,Please eat.,"Mange, s'il te plaît."
28171,You're very funny.,Vous êtes très drôles.
8220,Let's go away.,Allons-nous-en !


In [78]:
import re
# 특수문자의 띄어쓰기와 소문자화, 시작토큰과 종료토큰 삽입
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
   
    return sentence

In [79]:
# 이번에는 각각 \t와 \n을 사용하겠습니다. 두 토큰을 추가해줍니다.

# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
18297,Here's a blanket.,\t Voici une couverture. \n
2226,It's bogus.,\t C'est bidon. \n
19766,It's a nice town.,\t C'est une ville agréable. \n
21231,Tom loves to ski.,\t Tom adore le ski. \n
26901,Tom spoke quietly.,\t Tom a parlé doucement. \n


In [80]:
# 영어와 프랑스어를 array로 분리하여 변환

lines_eng= lines['eng'].to_numpy()
lines_fra= lines['fra'].to_numpy()
lines_eng

array(['Go.', 'Go.', 'Go.', ..., 'We also found this.',
       'We are busy people.', 'We are watching TV.'], dtype=object)

In [81]:
# 데이터 전처리

pre_eng = []
pre_fra = []

for eng, fra in zip(lines.eng, lines.fra):
    if len(eng) == 0: continue
    if len(fra) == 0: continue   
        
    pre_eng.append(preprocess_sentence(eng))
    pre_fra.append(preprocess_sentence(fra))

In [82]:
pre_eng[1:6]

['go .', 'go .', 'go .', 'hi .', 'hi .']

In [83]:
# 토큰화

eng_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[19, 4, 7], [19, 4, 7], [19, 4, 7]]

In [84]:
fra_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[9, 1, 19, 5, 1, 31, 1, 10],
 [9, 1, 15, 5, 12, 16, 28, 2, 13, 1, 10],
 [9, 1, 2, 7, 1, 12, 8, 11, 4, 2, 1, 31, 1, 10]]

In [85]:
# 단어장의 크기를 변수로 저장해줍니다. 0번 토큰을 고려하여 +1을 하고 저장해줍니다.

eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 51
프랑스어 단어장의 크기 : 73


In [86]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 19
프랑스어 시퀀스의 최대 길이 61


In [87]:
# 전체적인 통계 정보를 한꺼번에 출력해봅시다.

print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 51
프랑스어 단어장의 크기 : 73
영어 시퀀스의 최대 길이 19
프랑스어 시퀀스의 최대 길이 61


> 인코더의 입력으로 사용되는 영어 시퀀스와 달리, 프랑스어 시퀀스는 2가지 버전으로 나누어 준비해야 합니다. 하나는 디코더의 출력과 비교해야 할 정답 데이터로 사용해야 할 원래 목적에 따른 것입니다. 그리고 다른 하나는 이전 스텝에서 언급했던 교사 강요(Teacher forcing)을 위해 디코더의 입력으로 사용하기 위한 것입니다.

> 이때, 디코더의 입력으로 사용할 시퀀스는 < eos >토큰이 필요가 없고, 디코더의 출력과 비교할 시퀀스는 < sos >가 필요가 없기 때문입니다. 가령, 영어로 'I am a person'이라는 문장을 프랑스어 'Je suis une personne'로 번역하는 번역기를 만든다고 해봅시다. 훈련 과정에서 디코더는 '< sos > Je suis une personne'를 입력받아서 'Je suis une personne < eos >'를 예측하도록 훈련되므로, 이런 방식으로 생성된 두가지 버전의 시퀀스를 준비해야 합니다.

In [89]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [90]:
# 디코더의 입력과 출력을 각각 출력해봅시다.

print(decoder_input[:3])
print(decoder_target[:3])

[[9, 1, 19, 5, 1, 31, 1], [9, 1, 15, 5, 12, 16, 28, 2, 13, 1], [9, 1, 2, 7, 1, 12, 8, 11, 4, 2, 1, 31, 1]]
[[1, 19, 5, 1, 31, 1, 10], [1, 15, 5, 12, 16, 28, 2, 13, 1, 10], [1, 2, 7, 1, 12, 8, 11, 4, 2, 1, 31, 1, 10]]


디코더의 입력에선 숫자 10(\n)이 제거되었고 디코더의 출력에선 숫자 9(\t)가 제거되었다.

패딩을 진행하면 모든 샘플들의 길이가 정해준 길이로 동일하게 바뀐다. 여기서는 아까 저장해두었던 가장 긴 샘플의 길이인 max_eng_seq_len, max_fra_seq_len를 각각 사용한다.

In [91]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 19)
프랑스어 입력데이터의 크기(shape) : (33000, 61)
프랑스어 출력데이터의 크기(shape) : (33000, 61)


# 모델 훈련하기

## 데이터셋 분리

In [92]:
shuffled = np.arange(encoder_input.shape[0])
np.random.shuffle(shuffled)

In [93]:
shuffled

array([29749, 10705,  3244, ..., 22367, 26003, 31684])

In [94]:
encoder_input = encoder_input[shuffled]
decoder_input = decoder_input[shuffled]
decoder_target = decoder_target[shuffled]
encoder_input

array([[ 5,  1, 11, ..., 15, 10,  7],
       [ 5,  1, 21, ...,  0,  0,  0],
       [ 5,  1, 10, ...,  0,  0,  0],
       ...,
       [16,  4, 15, ...,  7,  0,  0],
       [ 3, 11,  6, ..., 19,  7,  0],
       [ 8, 10,  6, ..., 12, 16,  7]], dtype=int32)

In [101]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 19, 51)
프랑스어 입력데이터의 크기(shape) : (33000, 61, 73)
프랑스어 출력데이터의 크기(shape) : (33000, 61, 73)


각 문장들이 어떠한 순서에 따라 있으므로 랜덤하게 섞어서 데이터셋을 분리해야 한 쪽에 치우치지 않고 학습할 수 있다.

In [102]:
# 검증데이터 3000건 분리
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (33000, 19, 51)
프랑스어 학습 입력데이터의 크기(shape) : (33000, 61, 73)
프랑스어 학습 출력데이터의 크기(shape) : (33000, 61, 73)


## 임베딩 층 사용하기

In [129]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Masking

# 인코더에서 사용할 임베딩 층 사용 예시
embedding_size = 256
hidden_size = 256

encoder_inputs = Input(shape=(None, ))
enc_emb =  Embedding(eng_vocab_size, embedding_size,
                    input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(hidden_size, dropout = 0.5, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [130]:
# 디코더에서 사용할 임베딩 층
decoder_inputs = Input(shape=(None, ))
dec_emb =  Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(hidden_size, dropout = 0.5, return_sequences = True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

In [131]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

## 모델 구현하기

In [132]:
from tensorflow.keras.models import Model
import tensorflow as tf
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_20 (Embedding)       (None, None, 256)    13056       ['input_11[0][0]']               
                                                                                                  
 embedding_21 (Embedding)       (None, None, 256)    18688       ['input_12[0][0]']               
                                                                                           

In [133]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=64, epochs=50)

Epoch 1/50


ValueError: ignored

# Reference

https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html LSTM의 변수